In [43]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re  # Import regular expressions

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the provided URL
driver.get("https://www.goldenscent.com/en/c/fragrances.html?action=cat&id=3")

# Wait for the page to load
time.sleep(5)  # Adjust the wait time if needed

# Initialize an empty list to store unique hrefs
unique_hrefs = []

# Loop to collect unique hrefs for up to 200 perfumes
while len(unique_hrefs) < 5:
    # Locate the perfume links and extract their href attributes
    golden_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
    golden_hrefs = [link.get_attribute('href') for link in golden_links]

    # Collect unique hrefs
    for href in golden_hrefs:
        if href not in unique_hrefs:
            unique_hrefs.append(href)
        if len(unique_hrefs) >= 5:  # Stop once we have 200 unique hrefs
            break

   

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        rating_tag = soup.select_one('div.rc-midcontent h4')
        total_ratings_tag = soup.select_one('div.rc-midcontent span')
        img_tag = soup.select_one('div.thumbnails__block.active img')
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None
        
        if rating_tag:
            product_info['rating'] = rating_tag
        else:
            product_info['rating'] = None

        

        if total_ratings_tag:
            product_info['total_ratings'] = total_ratings_tag
            
        else:
            product_info['total_ratings']  = None

        if img_tag and 'src' in img_tag.attrs:
            product_info['img'] = img_tag['src']
            
        else:
            product_info['img']  = None
            

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)  
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)  
            product_info[label] = value  

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
# golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


                                               الاسم rating  total_ratings  \
0                   Yves Saint Laurent Libre Intense  [5.0]  [500 Ratings]   
1  Giorgio Armani Emporio Armani Stronger With Yo...  [4.9]  [500 Ratings]   
2                           Roberto Cavalli Paradiso  [4.9]  [500 Ratings]   
3          Very Important Notes Best Sellers Duo Set  [4.8]  [382 Ratings]   
4                   Versace Crystal Noir EDP - 90 ml  [4.8]    [6 Ratings]   

                                                 img  السعر النهائي  \
0  //assets.goldenscent.com/catalog/product/cache...            494   
1  //assets.goldenscent.com/catalog/product/cache...            536   
2  //assets.goldenscent.com/catalog/product/cache...            222   
3  //assets.goldenscent.com/catalog/product/cache...            249   
4  //assets.goldenscent.com/catalog/product/cache...            518   

                  Brand  Gender Product Type    Character Fragrance Family  \
0    Yves Saint Laurent   

In [44]:
golden_df

,الاسم,rating,total_ratings,img,السعر النهائي,Brand,Gender,Product Type,Character,Fragrance Family,Size,Ingredients,Concentration,Top Notes,Middle Notes,Base Notes,Product Color,Year of Launch,Category
0,Yves Saint Laurent Libre Intense,[5.0],[500 Ratings],//assets.goldenscent.com/catalog/product/cache...,494,Yves Saint Laurent,Women,Perfume,Extravagant,Fruity,50 ml,"Sugar, Raspberry, Orange Oil, Pink Jasmine, Am...",Eau de Parfum,-,-,-,NaN,NaN,NaN
1,Giorgio Armani Emporio Armani Stronger With Yo...,[4.9],[500 Ratings],//assets.goldenscent.com/catalog/product/cache...,536,Giorgio Armani,Men,Perfume,Charismatic,Aromatic,100 ml,"Spices, Violet, Lavender, Sweet Toffee, Carame...",Eau de Parfum,"Pink pepper, Juniper, Violet leaf","Lavender, Sage, Toffee, Cinnamon","Tonka bean, Suede, Amber, Vanilla",Gold,2019,NaN
2,Roberto Cavalli Paradiso,[4.9],[500 Ratings],//assets.goldenscent.com/catalog/product/cache...,222,Roberto Cavalli,Women,Perfume,Charismatic,Woody,50 ml,"Citrus, mandarin, bergamot, jasmine, pine, cyp...",Eau de Parfum,"Citruses , Mandarin , Bergamot",Jasmine,"Cypress, Parasol pine, Pink laurel",NaN,2015,Perfume
3,Very Important Notes Best Sellers Duo Set,[4.8],[382 Ratings],//assets.goldenscent.com/catalog/product/cache...,249,Very Important Notes,Unisex,Bundles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Versace Crystal Noir EDP - 90 ml,[4.8],[6 Ratings],//assets.goldenscent.com/catalog/product/cache...,518,Versace,Women,Perfume,Modern,Floral Oriental,90 ml,"Ginger, cardamom, pepper, orange buds, peony, ...",Eau de Parfum,"Ginger, cardamom, pepper","orange buds, peony, gardenia, coconut","sandalwood, musk, amber.",NaN,2004,NaN


In [13]:
golden_df[golden_df["نوع المنتج"] == "عطور"].isnull().sum().sort_values(ascending = False)

مفتاح البحث        708
نوع المكياج        707
اللون              635
Category           628
سنة الإصدار        354
المكونات           322
قاعدة العطر        191
قلب العطر          190
مقدمة العطر        188
شخصية عطرك          22
العائلة العطرية     17
الماركة              6
نسبة التركيز         2
الحجم                2
السعر النهائي        0
نوع المنتج           0
الجنس                0
الاسم                0
dtype: int64

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load links from the file
with open("unique_links.txt", "r", encoding="utf-8") as file:
    unique_hrefs = [line.strip() for line in file]

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            if final_price:
                product_info["السعر النهائي"] = int(final_price)
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)
            product_info[label] = value

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes2.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


                                                  الاسم  السعر النهائي  \
0                        كورلوف كوير ميثيك أو دو برفيوم           1116   
1                              كالفين كلاين ايفوريا ومن            298   
2         قراس للعطور عود ذا بالاس أو دو برفيوم - 30 مل            159   
3                                  ماركو سيروسي هارموني            333   
4                                 لورينزو فيلوريسي أومو            629   
...                                                 ...            ...   
8139               سلفاتوري فيراغامو توسكان سويل كونفوا            754   
8140                                   مونتال داي دريمز            426   
8141                  شكسبير لوفرز كومبلين أو دو برفيوم            569   
8142                       هتان مسك الرمان أو دو برفيوم            140   
8143  جويو يونيك أند سينسوريل بارفيومز سوبودراج أو د...            814   

                               الماركة        الجنس نوع المنتج    شخصية عطرك  \
0                              

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load links from the file
with open("unique_links.txt", "r", encoding="utf-8") as file:
    unique_hrefs = [line.strip() for line in file]

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        rating_tag = soup.select_one('div.rc-midcontent h4')
        total_ratings_tag = soup.select_one('div.rc-midcontent span')
        img_tag = soup.select_one('div.thumbnails__block.active img')
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None
        
        if rating_tag:
            product_info['rating'] = rating_tag
        else:
            product_info['rating'] = None

        

        if total_ratings_tag:
            product_info['total_ratings'] = total_ratings_tag
            
        else:
            product_info['total_ratings']  = None

        if img_tag and 'src' in img_tag.attrs:
            product_info['img'] = img_tag['src']
            
        else:
            product_info['img']  = None
            

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)  
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)  
            product_info[label] = value  

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

# Save the DataFrame to a CSV file
# golden_df.to_csv("golden_scent_perfumes.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()

                                                  الاسم  السعر النهائي  \
0                        كورلوف كوير ميثيك أو دو برفيوم           1116   
1                              كالفين كلاين ايفوريا ومن            298   
2         قراس للعطور عود ذا بالاس أو دو برفيوم - 30 مل            159   
3                                  ماركو سيروسي هارموني            333   
4                                 لورينزو فيلوريسي أومو            629   
...                                                 ...            ...   
8139               سلفاتوري فيراغامو توسكان سويل كونفوا            754   
8140                                   مونتال داي دريمز            426   
8141                  شكسبير لوفرز كومبلين أو دو برفيوم            569   
8142                       هتان مسك الرمان أو دو برفيوم            140   
8143  جويو يونيك أند سينسوريل بارفيومز سوبودراج أو د...            814   

                               الماركة        الجنس نوع المنتج    شخصية عطرك  \
0                              

In [2]:
golden_df

,الاسم,السعر النهائي,الماركة,الجنس,نوع المنتج,شخصية عطرك,العائلة العطرية,الحجم,سنة الإصدار,نسبة التركيز,...,قاعدة العطر,Category,المكونات,اللون,النوتات العُليا,النوتات المتوسطة,النوتات القاعدية,التركيبة,نوع المكياج,مفتاح البحث
0,كورلوف كوير ميثيك أو دو برفيوم,1116,كورلوف,رجالي/نسائي,عطور,كلاسيكية,جلدية,100 مل,2018,أو دو برفيوم,...,الجلود,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,كالفين كلاين ايفوريا ومن,298,كالفن كلاين,نسائي,عطور,حيوية,فاكهية,50 مل,2005,أو دو برفيوم,...,عنبر ، خشب,عطور,رمان ، لوتس بلوسوم ، بلاك أوركيد ، عنبر ، خشب,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,قراس للعطور عود ذا بالاس أو دو برفيوم - 30 مل,159,قراس للعطور,رجالي/نسائي,عطور,كلاسيكية,جلدية,30 مل,NaN,أو دو برفيوم,...,الجلود، أخشاب الصندل,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ماركو سيروسي هارموني,333,ماركو سيروسي,رجالي,عطور,جذابة,شرقية خشبية,100 مل,NaN,أو دو تواليت,...,باتشولي، نجيل الهند.,NaN,الفلفل الأسود، هيل، كمون، جوزة الطيب، زهر البر...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,لورينزو فيلوريسي أومو,629,لورينزو فيلوريسي فيرنزي,رجالي,عطور,حيوية,أروماتيك,100 مل,1993,أو دو تواليت,...,خشب الصندل، الباتشولي، المسك، نجيل الهند، طحلب...,NaN,أوراق الغار، البتيتغرين، البرغموت، الليمون الأ...,"أزرق,فضي",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8139,سلفاتوري فيراغامو توسكان سويل كونفوا,754,سلفاتوري فيراغامو,رجالي/نسائي,عطور,محبة للطبيعة,حمضية,75 مل,2013,أو دو تواليت,...,امبروكسان، الكشمير,NaN,بذور الجزر، الجريب فروت، السرو، الياسمين، امبر...,متعدد الألوان,NaN,NaN,NaN,NaN,NaN,NaN
8140,مونتال داي دريمز,426,مونتال,رجالي/نسائي,عطور,حيوية,زهرية,100 مل,2017,أو دو برفيوم,...,الفانيليا، جوز الهند,NaN,برتقال الماندرين، زهر البرتقال، زهرة تياري، ال...,"أزرق,ذهبي",NaN,NaN,NaN,NaN,NaN,NaN
8141,شكسبير لوفرز كومبلين أو دو برفيوم,569,شكسبير برفيوم,رجالي/نسائي,عطور,حيوية,أروماتيك,100 مل,2018,أو دو برفيوم,...,NaN,NaN,التوت، فول التونكا، جلد، العنبر,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8142,هتان مسك الرمان أو دو برفيوم,140,هتان,رجالي/نسائي,عطور,حيوية,فاكهية,50 مل,NaN,أو دو برفيوم,...,NaN,NaN,حبوب التونكا، الفانيليا، الرمان.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
golden_df[golden_df["نوع المنتج"] == "عطور"].isnull().sum().sort_values(ascending = False)

In [45]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load links from the file
with open("updated_links_with_en.txt", "r", encoding="utf-8") as file:
    unique_hrefs = [line.strip() for line in file]

# Initialize a list to store perfume details with both name and price
perfume_details = []

# Loop through each unique href and navigate to it
for index, href in enumerate(unique_hrefs):
    try:
        driver.get(href)  # Navigate to the perfume detail page using the href
        time.sleep(5)  # Wait for the perfume details page to load

        # Use BeautifulSoup on the page source loaded by Selenium
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Initialize a dictionary to store the perfume information
        product_info = {}

        # Extract the name
        name_element = soup.find('div', {'class': "col-xs-12"})
        rating_tag = soup.select_one('div.rc-midcontent h4')
        total_ratings_tag = soup.select_one('div.rc-midcontent span')
        img_tag = soup.select_one('div.thumbnails__block.active img')
        if name_element:
            name = name_element.find('h1').get_text(strip=True)
            product_info['الاسم'] = name
        else:
            product_info['الاسم'] = None
        
        if rating_tag:
            product_info['rating'] = rating_tag
        else:
            product_info['rating'] = None

        

        if total_ratings_tag:
            product_info['total_ratings'] = total_ratings_tag
            
        else:
            product_info['total_ratings']  = None

        if img_tag and 'src' in img_tag.attrs:
            product_info['img'] = img_tag['src']
            
        else:
            product_info['img']  = None
            

        # Extract the final price (discounted or regular)
        price_content = soup.find('div', {'class': 'price-content'})
        final_price = None

        if price_content:
            # Check if there is a discount (look for 'savings' class)
            savings_div = price_content.find('div', {'class': 'savings'})
            if savings_div:
                # Extract the discounted price from the second span within 'savings'
                spans = savings_div.find_all('span')
                if len(spans) > 1:
                    final_price = re.search(r'\d+', spans[1].get_text(strip=True)).group()
            else:
                # If there's no discount, use the regular price in 'special'
                special_price = price_content.find('span', {'class': 'special'})
                if special_price:
                    final_price = re.search(r'\d+', special_price.get_text(strip=True)).group()

            # Store the final price if found
            if final_price:
                product_info["السعر النهائي"] = int(final_price)  # Convert to integer for consistency
            else:
                product_info["السعر النهائي"] = None
        else:
            print(f"Price not found on perfume {index + 1}")

        # Extract additional attributes
        rows = soup.find_all('li', {'class': 'row'})
        for row in rows:
            label = row.find('div', {'class': 'col-md-2 p0'}).get_text(strip=True)  
            value = row.find('div', {'class': 'col-md-5 p0'}).get_text(strip=True)  
            product_info[label] = value  

        # Append the perfume information to the list
        perfume_details.append(product_info)

    except Exception as e:
        print(f"Error on perfume {index + 1}: {e}")

# Convert the list of perfume details to a DataFrame
golden_df = pd.DataFrame(perfume_details)

# Display the DataFrame with perfume names and final prices
print(golden_df)

#Save the DataFrame to a CSV file
golden_df.to_csv("golden_scent_perfumes3.csv", index=False, encoding="utf-8-sig")

# Close the browser
driver.quit()


Price not found on perfume 4149
Price not found on perfume 5801
Price not found on perfume 7762
                                                الاسم rating  total_ratings  \
0                           Korloff Cuir Mythique EDP    [0]    [0 Ratings]   
1                         Calvin Klein Euphoria Women  [4.8]  [500 Ratings]   
2          Grasse Perfumes Oud The Palace EDP - 30 ml  [4.0]    [1 Ratings]   
3                               Marco Serussi Harmony    [0]    [0 Ratings]   
4                              Lorenzo Villoresi Uomo  [5.0]    [2 Ratings]   
...                                               ...    ...            ...   
8139           Salvatore Ferragamo Tuscan Soul Convio    [0]    [0 Ratings]   
8140                               Montale Day Dreams  [4.7]    [3 Ratings]   
8141                 Shakespeare Lover's Complain EDP  [5.0]    [1 Ratings]   
8142                       Hatan Pomegranate Musk EDP    [0]    [0 Ratings]   
8143  Joyau Unique & Sensoriel Parf

In [46]:
golden_df

,الاسم,rating,total_ratings,img,السعر النهائي,Brand,Gender,Product Type,Character,Fragrance Family,...,Concentration,Top Notes,Middle Notes,Base Notes,Category,Ingredients,Product Color,Preferences,Makeup Type,Search Key
0,Korloff Cuir Mythique EDP,[0],[0 Ratings],//assets.goldenscent.com/catalog/product/3/7/3...,1116.0,Korloff,Unisex,Perfume,Classical,Leather,...,Eau de Parfum,"Mandarin, Pine, Juniper","Ginger, Galbanum, Iris",Leather,NaN,NaN,NaN,NaN,NaN,NaN
1,Calvin Klein Euphoria Women,[4.8],[500 Ratings],//assets.goldenscent.com/catalog/product/cache...,298.0,Calvin Klein,Women,Perfume,Extravagant,Fruity,...,Eau de Parfum,Pomegranate,"Lotus Blossom, Black Orchid","Amber, Wood",Perfume,"Pomegranate, Lotus Blossom, Black Orchid, Ambe...",NaN,NaN,NaN,NaN
2,Grasse Perfumes Oud The Palace EDP - 30 ml,[4.0],[1 Ratings],//assets.goldenscent.com/catalog/product/cache...,159.0,Grasse Perfumes,Unisex,Perfume,Classical,Leather,...,Eau de Parfum,"Rose Oil, Bulgarian Rose",Cinnamon,"Leather, Sandalwood",NaN,NaN,NaN,NaN,NaN,NaN
3,Marco Serussi Harmony,[0],[0 Ratings],//assets.goldenscent.com/catalog/product/3/7/3...,333.0,Marco Serussi,Men,Perfume,Charismatic,Woody Oriental,...,Eau de Toilette,Black Pepper- Cardamom- Cumin,Nutmeg- Neroli- Melon.,Patchouli- Vetiver.,NaN,"Black Pepper, Cardamom, Cumin, Nutmeg, Neroli,...",NaN,NaN,NaN,NaN
4,Lorenzo Villoresi Uomo,[5.0],[2 Ratings],//assets.goldenscent.com/catalog/product/g/o/g...,629.0,Lorenzo Villoresi Firenze,Men,Perfume,Extravagant,Aromatic,...,Eau de Toilette,"Bergamot, Petitgrain, Laurel, Lemon, Orange, C...","Neroli, Sage, Juniper, Clove, Pepper, Thyme","Oakmoss, Musk, Patchouli, Rosemary, Sandalwood...",NaN,"Bergamot, Petitgrain, Laurel, Lemon, Orange, C...","Blue,Silver",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8139,Salvatore Ferragamo Tuscan Soul Convio,[0],[0 Ratings],//assets.goldenscent.com/catalog/product/8/0/8...,754.0,Salvatore Ferragamo,Unisex,Perfume,Natural,Citrus,...,Eau de Toilette,"Grapefruit, Carrot Seed","Cypress, Jasmine Petals","Ambroxan, Cashmere",NaN,"carrot seeds, grapefruit, jasmine, cypress tre...",Multi,NaN,NaN,NaN
8140,Montale Day Dreams,[4.7],[3 Ratings],//assets.goldenscent.com/catalog/product/cache...,426.0,Montale,Unisex,Perfume,Extravagant,Floral,...,Eau de Parfum,"Orange Blossom, Jasmine, Mandarin","Neroli, Tiare, Sandalwood","Vanilla, Coconut",NaN,"Mandarin Orange, Orange Blossom, Neroli, Tiare...","Blue,Gold",NaN,NaN,NaN
8141,Shakespeare Lover's Complain EDP,[5.0],[1 Ratings],//assets.goldenscent.com/catalog/product/3/0/3...,569.0,Shakespeare Perfume,Unisex,Perfume,Extravagant,Aromatic,...,Eau de Parfum,NaN,NaN,NaN,NaN,"Berry, Tonka Bean, Leather, Amber",NaN,NaN,NaN,NaN
8142,Hatan Pomegranate Musk EDP,[0],[0 Ratings],//assets.goldenscent.com/catalog/product/cache...,140.0,Hatan,Unisex,Perfume,Extravagant,Fruity,...,Eau de Parfum,NaN,NaN,NaN,NaN,"Tonka Bean, Vanilla, Pomegranate",NaN,NaN,NaN,NaN
